<a href="https://colab.research.google.com/github/onleey/python_demo/blob/master/407_kakao%EB%A9%94%EC%84%B8%EC%A7%80_%EA%B8%B0%ED%9B%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd  '/content/drive/MyDrive/001'

Mounted at /content/drive
/content/drive/MyDrive/001


In [24]:
from bs4 import BeautifulSoup
import requests

import json
import datetime
import kakao_utils

KAKAO_TOKEN_FILENAME = 'res/kakao_message/kakao_token.json'

#rest api 앱키를 입력
KAKAO_APP_KEY ='코드입력하세요'


#토큰업데이트하기
tokens= kakao_utils.update_tokens(KAKAO_APP_KEY,KAKAO_TOKEN_FILENAME)


result = {}

url = 'https://www.weather.go.kr/w/obs-climate/land/city-obs.do'
res = requests.get(url)
print(res)

soup = BeautifulSoup(res.text,'html.parser')

datetm = soup.select_one('#select-tm').get('value')
print(datetm)
result['datatm'] = datetm

tbody_tag = soup.find('tbody')
tr_rs_tag = tbody_tag.find_all('tr')


for tr_tag in tr_rs_tag:
  td_rs_tag = tr_tag.find_all('td')

  city = td_rs_tag[0].string
  if city == '서울':
    cur_temp = td_rs_tag[5].string
    sen_temp = td_rs_tag[7].string

    result['city'] = city
    result['cur_temp'] = cur_temp
    result['sen_temp'] = sen_temp
    break

print(result)

#날씨정보 만들기

text = f"""\
#날씨정보 ({result['datatm']})
도시 : {result['city']}
현재온도 :{result['cur_temp']}
체감온도 : {result['sen_temp']}
"""

#텍스트 템플릿 형식 만들기

weather_url='https://www.weather.go.kr/w/obs-climate/land/city-obs.do?tm=2023.12.6.12:00&type=t99&mode=0&reg=109&auto_man=m&stn=108'

template = {

         'object_type':'text',
         'text':text,
         'link':{
             'web_url': weather_url,
             'mobile_web_url': weather_url
         },

         'button_title': '날씨 상세보기'
      }



#카카오톡 메세지 보내기
res = kakao_utils.send_message(KAKAO_TOKEN_FILENAME,template)

if res.json().get('result_code') == 0:
  print('날씨 및 미세먼지 정보 성공적으로 보냈습니다.')

else:
  print('날씨 및 미세먼지 정보 성공적으로 보내지 못했습니다. 오류메시지:',res.json())






{'access_token': 'iLNmCmw0Tfq91ul3nCFnLVYrDeQob4WXBHoKKw0gAAABjD1MRa9b9Pmr5eg_ZA', 'token_type': 'bearer', 'expires_in': 21599}
<Response [200]>
2023.12.06.13:00
{'datatm': '2023.12.06.13:00', 'city': '서울', 'cur_temp': '8.0', 'sen_temp': '6.4'}
날씨 및 미세먼지 정보 성공적으로 보냈습니다.
